In [17]:
%env SPARK_VERSION=3.2 # in cmd >pyspark --version  - to check version 

env: SPARK_VERSION=3.2 # in cmd >pyspark --version  - to check version


In [18]:
import pydeequ
from pyspark.sql import SparkSession, Row

In [19]:
server_url = "jdbc:sqlserver://host.docker.internal" 
database_name = "TRN"
url = server_url + ";" + "databaseName=" + database_name + ";trustServerCertificate=True;"

table = "hr.employees"
user = "oleksandraLogin" # your user creds here
password  = "StrongPassword1@" # your user creds here

In [20]:
# TODO connect to DB with Spark using JDBC connection to read the data

connection_details = { "user": user, "password": password, "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver", }

spark = (SparkSession
    .builder
    .config("spark.jars.packages","com.amazon.deequ:deequ:1.2.2-spark-3.1")
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.extraClassPath", "/home/jovyan/mssql-jdbc-12.6.1.jre11.jar")
    .getOrCreate())


#mssql-jdbc-12.6.1.jre11.jar were uploaded from
# https://learn.microsoft.com/en-us/sql/connect/jdbc/download-microsoft-jdbc-driver-for-sql-server?view=sql-server-ver16#download
# -> unzip 
# -> copied into podmman container by using import file from jupiter notebook
# -> comand >podman cp mssql-jdbc-12.6.1.jre11.jar eb28ed05474b:/home/jovyan/work
# or comand  >podman machine ssh sh -c 'cat > //home/jovyan/work/ mssql-jdbc-12.6.1.jre11.jar < mssql-jdbc-12.6.1.jre11.jar 
# doesn't work. As reason: I use Windows cmd. Podman works with Linuks
# for future try to use Linux VM

In [21]:
df = spark.read.jdbc(url=url, table=table, properties=connection_details)

In [22]:
#df.printSchema()  # test comand

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- job_id: integer (nullable = true)
 |-- salary: decimal(8,2) (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [23]:
#df.show(10,False)   # test comand

+-----------+----------+---------+--------------------------------+------------+----------+------+--------+----------+-------------+
|employee_id|first_name|last_name|email                           |phone_number|hire_date |job_id|salary  |manager_id|department_id|
+-----------+----------+---------+--------------------------------+------------+----------+------+--------+----------+-------------+
|100        |Steven    |King     |steven.king@sqltutorial.org     |515.123.4567|1987-06-17|4     |24000.00|null      |9            |
|101        |Neena     |Kochhar  |neena.kochhar@sqltutorial.org   |515.123.4568|1989-09-21|5     |17000.00|100       |9            |
|102        |Lex       |De Haan  |lex.de haan@sqltutorial.org     |515.123.4569|1993-01-13|5     |17000.00|100       |9            |
|103        |Alexander |Hunold   |alexander.hunold@sqltutorial.org|590.423.4567|1990-01-03|9     |9000.00 |102       |6            |
|104        |Bruce     |Ernst    |bruce.ernst@sqltutorial.org     |59

In [24]:
### Data Analyzers section

from pydeequ.analyzers import *
analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Completeness("employee_id"))\
                    .addAnalyzer(Completeness("email")) \
                    .addAnalyzer(Completeness("phone_number")) \
                    .addAnalyzer(CountDistinct("employee_id")) \
                    .addAnalyzer(Distinctness("employee_id")) \
                    .addAnalyzer(Distinctness("phone_number")) \
                    .addAnalyzer(Distinctness("first_name")) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Mean("salary")) \
                    .addAnalyzer(Maximum("salary")) \
                    .addAnalyzer(Minimum("salary")) \
                    .addAnalyzer(Compliance("salary", "salary > 0")) \
                    .run() 
                   
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+------------+-------------+-------+
| entity|    instance|         name|  value|
+-------+------------+-------------+-------+
| Column| employee_id| Distinctness|    1.0|
| Column|      salary|   Compliance|    1.0|
| Column| employee_id| Completeness|    1.0|
| Column|       email| Completeness|    1.0|
| Column| employee_id|CountDistinct|   40.0|
|Dataset|           *|         Size|   40.0|
| Column|  first_name| Distinctness|  0.925|
| Column|phone_number| Completeness|   0.85|
| Column|      salary|         Mean| 8060.0|
| Column|      salary|      Maximum|24000.0|
| Column|      salary|      Minimum| 2500.0|
| Column|phone_number| Distinctness|    1.0|
+-------+------------+-------------+-------+



In [25]:
#!spark-submit --version   #check spark version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.13
Branch HEAD
Compiled by user ubuntu on 2021-10-06T12:46:30Z
Revision 5d45a415f3a29898d92380380cfd82bfc7f579ea
Url https://github.com/apache/spark
Type --help for more information.


In [26]:
### Data profiling section
# At the moment exist error. Need to synchronize versions on spark and deeequ

from pydeequ.profiles import *
result = ColumnProfilerRunner(spark) \
     .onData(df) \
     .run()

for col, profile in result.profiles.items():
    print(profile)

Py4JJavaError: An error occurred while calling o268.run.
: java.lang.NoSuchMethodError: 'org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression org.apache.spark.sql.catalyst.expressions.aggregate.AggregateFunction.toAggregateExpression(boolean)'
	at org.apache.spark.sql.DeequFunctions$.withAggregateFunction(DeequFunctions.scala:31)
	at org.apache.spark.sql.DeequFunctions$.stateful_approx_count_distinct(DeequFunctions.scala:60)
	at com.amazon.deequ.analyzers.ApproxCountDistinct.aggregationFunctions(ApproxCountDistinct.scala:52)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.$anonfun$runScanningAnalyzers$3(AnalysisRunner.scala:319)
	at scala.collection.immutable.List.flatMap(List.scala:366)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.liftedTree1$1(AnalysisRunner.scala:319)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.runScanningAnalyzers(AnalysisRunner.scala:318)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.doAnalysisRun(AnalysisRun

In [ ]:
### Constraint Suggestions section
# TODO find meaninful constraints here

In [45]:
from pydeequ.suggestions import *
# At the moment exist error. Need to synchronize versions on spark and deeequ

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions
print(json.dumps(suggestionResult, indent=2))

Py4JJavaError: An error occurred while calling o536.run.
: java.lang.NoSuchMethodError: 'org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression org.apache.spark.sql.catalyst.expressions.aggregate.AggregateFunction.toAggregateExpression(boolean)'
	at org.apache.spark.sql.DeequFunctions$.withAggregateFunction(DeequFunctions.scala:31)
	at org.apache.spark.sql.DeequFunctions$.stateful_approx_count_distinct(DeequFunctions.scala:60)
	at com.amazon.deequ.analyzers.ApproxCountDistinct.aggregationFunctions(ApproxCountDistinct.scala:52)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.$anonfun$runScanningAnalyzers$3(AnalysisRunner.scala:319)
	at scala.collection.immutable.List.flatMap(List.scala:366)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.liftedTree1$1(AnalysisRunner.scala:319)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.runScanningAnalyzers(AnalysisRunner.scala:318)
	at com.amazon.deequ.analyzers.runners.AnalysisRunner$.doAnalysisRun(AnalysisRun

In [28]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report

In [42]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check \
        .isComplete("email")  \
        .isComplete("salary") \
        .isUnique("employee_id")  \
        .hasMax("employee_id", lambda x: x == 40) \
        .isNonNegative("salary")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+----------------------------------------------------------------------------------------------------------------+-----------------+------------------------------------------------------+
|check       |check_level|check_status|constraint                                                                                                      |constraint_status|constraint_message                                    |
+------------+-----------+------------+----------------------------------------------------------------------------------------------------------------+-----------------+------------------------------------------------------+
|Review Check|Warning    |Warning     |CompletenessConstraint(Completeness(email,None))                                                                |Success          |                                                      |
|Review Check|Warning    |Warning     |CompletenessConstraint(Completeness(salary,None))        

In [43]:
checkResult_pandas_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas=True)
checkResult_pandas_df

,entity,instance,name,value
0,Column,employee_id,Uniqueness,1.0
1,Column,employee_id,Maximum,206.0
2,Column,salary is non-negative,Compliance,1.0
3,Column,email,Completeness,1.0
4,Column,salary,Completeness,1.0


In [46]:
#inmort result of Constraint Verification section into csv
import pandas as pd

result=checkResult_df.toPandas().to_csv("result.csv")